In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hitters/Hitters.csv


In [6]:
from warnings import filterwarnings
filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split,cross_val_score,cross_val_predict,ShuffleSplit,GridSearchCV
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.preprocessing import scale
from sklearn import model_selection
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor,RandomForestRegressor,BaseEnsemble,GradientBoostingRegressor
import xgboost 
from xgboost import XGBRegressor
from sklearn.svm import SVR

pip install skompiler

In [12]:
conda install -c conda-forge lightgbm

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.1.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - lightgbm


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    joblib-1.2.0               |     pyhd8ed1ab_0         205 KB  conda-forge
    lightgbm-3.1.1             |   py37hcd2ae1e_0         1.6 MB  conda-forge
    scikit-learn-1.0.2         |   py37hf9e9bfc_0         7.8 MB  conda-forge
    threadpoolctl-3.1.0        |     pyh8a188c0_0          18 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         9.6 MB

The following NEW packages will be INSTALLED:

  joblib             conda-forge/noarch::joblib-1.2.0-p

In [1]:
from lightgbm import LGBMRegressor

In [2]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [3]:
from catboost import CatBoostRegressor

In [11]:
#KNN
hts = pd.read_csv("/kaggle/input/hitters/Hitters.csv")
hts.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N


In [13]:
hts.dropna(inplace=True)

In [14]:
one_hot_encoded = pd.get_dummies(hts[["League","Division","NewLeague"]])
one_hot_encoded.head()

,League_A,League_N,Division_E,Division_W,NewLeague_A,NewLeague_N
1,0,1,0,1,0,1
2,1,0,0,1,1,0
3,0,1,1,0,0,1
4,0,1,1,0,0,1
5,1,0,0,1,1,0


In [15]:
new_hts = hts.drop(["League","Division","NewLeague","Salary"],axis=1).astype("float64")

In [17]:
x = pd.concat([new_hts,one_hot_encoded[["League_N","Division_W","NewLeague_N"]]],axis=1)
x.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,League_N,Division_W,NewLeague_N
1,315.0,81.0,7.0,24.0,38.0,39.0,14.0,3449.0,835.0,69.0,321.0,414.0,375.0,632.0,43.0,10.0,1,1,1
2,479.0,130.0,18.0,66.0,72.0,76.0,3.0,1624.0,457.0,63.0,224.0,266.0,263.0,880.0,82.0,14.0,0,1,0
3,496.0,141.0,20.0,65.0,78.0,37.0,11.0,5628.0,1575.0,225.0,828.0,838.0,354.0,200.0,11.0,3.0,1,0,1
4,321.0,87.0,10.0,39.0,42.0,30.0,2.0,396.0,101.0,12.0,48.0,46.0,33.0,805.0,40.0,4.0,1,0,1
5,594.0,169.0,4.0,74.0,51.0,35.0,11.0,4408.0,1133.0,19.0,501.0,336.0,194.0,282.0,421.0,25.0,0,1,0


In [18]:
y=hts.Salary

In [20]:
hts.shape

(263, 20)

In [21]:
x.shape

(263, 19)

In [22]:
y.shape

(263,)

In [23]:
x_train = x.iloc[:210]
x_test = x.iloc[210:]
y_train = y[:210]
y_test = y[210:]
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(210, 19)
(53, 19)
(210,)
(53,)


In [25]:
k_nearest_neighbor = KNeighborsRegressor().fit(x_train,y_train)

k_nearest_neightbor

In [26]:
k_nearest_neighbor.n_neighbors

5

In [27]:
k_nearest_neighbor.metric

'minkowski'

In [28]:
y_pred = k_nearest_neighbor.predict(x_train)

In [29]:
np.sqrt(mean_squared_error(y_train,y_pred))

272.62826664397875

In [31]:
r2_score(y_train,y_pred)

0.6566334622730783

In [33]:
y_pred=k_nearest_neighbor.predict(x_test)

In [35]:
np.sqrt(mean_squared_error(y_test, y_pred))

284.2568747067342

In [36]:
r2_score(y_test,y_pred)

0.44677834961156204

In [37]:
Root_Mean_square_Errors= []

for i in range(15):
    i+=1
    knn=KNeighborsRegressor(n_neighbors=i).fit(x_train,y_train)
    y_pred = knn.predict(x_train)
    rmse=np.sqrt(mean_squared_error(y_train,y_pred))
    Root_Mean_square_Errors.append(rmse)
    print("RMSE value for k=",i,":",rmse)

RMSE value for k= 1 : 0.0
RMSE value for k= 2 : 180.88345020122767
RMSE value for k= 3 : 234.31341582447726
RMSE value for k= 4 : 253.85066788405706
RMSE value for k= 5 : 272.62826664397875
RMSE value for k= 6 : 290.4048911727775
RMSE value for k= 7 : 297.25110879862706
RMSE value for k= 8 : 310.8280894916466
RMSE value for k= 9 : 318.03658609122976
RMSE value for k= 10 : 318.4999056344273
RMSE value for k= 11 : 324.15431254360897
RMSE value for k= 12 : 329.04827672314815
RMSE value for k= 13 : 329.66310104255126
RMSE value for k= 14 : 332.4164560588913
RMSE value for k= 15 : 335.44623573055964
